In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('sentiments.csv')
df.head()

,text,score
0,"Сериал очень люблю, но Академия и Земля вызыва...",6
1,"думал, что будет лучше идея очень интересна - ...",7
2,с творчеством Головачева я познакомился посред...,10
3,"то-то я и в большое неудовольствие прочитал ""А...",5
4,как мне показалось местами сильно смахивает на...,6


In [3]:
from sklearn.model_selection import train_test_split

In [4]:
class SimpleDataSplitter():

    def __init__(self, text_attr, rate_attr, test_part_size=.3):
        self.text_attr = text_attr
        self.rate_attr = rate_attr
        self.test_part_size = test_part_size
    
    def split_data(self, data):
        x = data[self.text_attr]
        y = data[self.rate_attr]

        x_train, x_test, y_train, y_test = train_test_split(
            x, y, test_size = self.test_part_size)

        return x_train, x_test, y_train, y_test 

In [11]:
text_attr = 'text'
rate_attr = 'score'
splitter = SimpleDataSplitter(text_attr, rate_attr, test_part_size=.3)
X_train, X_test, y_train, y_test = splitter.split_data(df)


In [19]:
my_tags = [str(x) for x in range(1,11)]

In [15]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline
import time
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report

In [20]:
%%time

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)



y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

/home/sultan/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/sultan/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


accuracy 0.2911666666666667
              precision    recall  f1-score   support

           1       0.18      0.07      0.11       107
           2       0.06      0.05      0.06        92
           3       0.14      0.06      0.08       179
           4       0.09      0.06      0.07       230
           5       0.17      0.15      0.16       377
           6       0.18      0.12      0.14       341
           7       0.15      0.11      0.13       583
           8       0.22      0.26      0.23       999
           9       0.28      0.32      0.30      1419
          10       0.44      0.50      0.47      1673

    accuracy                           0.29      6000
   macro avg       0.19      0.17      0.17      6000
weighted avg       0.27      0.29      0.28      6000

CPU times: user 1min 27s, sys: 1min 43s, total: 3min 10s
Wall time: 55.8 s


In [21]:
from nltk.corpus import stopwords
STOPWORDS = stopwords.words('russian')

In [23]:
import re, string 
import pandas as pd   
from collections import defaultdict
import spacy
from sklearn.manifold import TSNE

from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
%matplotlib inline


def clean_text(text):
    '''Make text lowercase, remove square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub(r'[\[\]\(\)\{\}]', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    
    text = word_tokenize(text)
    # Remove a sentence if it is only one word long
    if len(text) > 2:
        return [word for word in text if word not in STOPWORDS]

tokenized_corpus = df['text'].apply(lambda x: clean_text(x))
print(tokenized_corpus)

0        [сериал, очень, люблю, академия, земля, вызыва...
1        [думал, идея, очень, интересна, города, иные, ...
2        [творчеством, головачева, познакомился, посред...
3        [тото, большое, неудовольствие, прочитал, арту...
4        [показалось, местами, сильно, смахивает, посла...
                               ...                        
19995    [оставляет, место, разгадывания, загадок, хотя...
19996    [сериал, тех, поиске, квики, секса, обязательс...
19997    [вообщето, детективы, боевики, люблю, мелодрам...
19998    [изменить, жизнь, помощью, джакузи, плохая, ид...
19999    [шимпанзе, играет, пакмэна, фильме, такова, ув...
Name: text, Length: 20000, dtype: object


In [37]:
df_clean = pd.DataFrame(tokenized_corpus)

In [38]:
df_clean

,text
0,"[сериал, очень, люблю, академия, земля, вызыва..."
1,"[думал, идея, очень, интересна, города, иные, ..."
2,"[творчеством, головачева, познакомился, посред..."
3,"[тото, большое, неудовольствие, прочитал, арту..."
4,"[показалось, местами, сильно, смахивает, посла..."
...,...
19995,"[оставляет, место, разгадывания, загадок, хотя..."
19996,"[сериал, тех, поиске, квики, секса, обязательс..."
19997,"[вообщето, детективы, боевики, люблю, мелодрам..."
19998,"[изменить, жизнь, помощью, джакузи, плохая, ид..."


In [35]:
'''
# This lemmitizer is not usefull for russian language
import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatizer(text): 
    
    sent = []
    if text != None:
        for word in text:
            sent.append(wordnet_lemmatizer.lemmatize(word))
    return sent

df_clean["text_lemmatize"] =  df_clean.apply(lambda x: lemmatizer(x['text']), axis=1)
'''

In [43]:
df_clean = df_clean.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

In [44]:
df_clean

,text
0,"[сериал, очень, люблю, академия, земля, вызыва..."
1,"[думал, идея, очень, интересна, города, иные, ..."
2,"[творчеством, головачева, познакомился, посред..."
3,"[тото, большое, неудовольствие, прочитал, арту..."
4,"[показалось, местами, сильно, смахивает, посла..."
...,...
19995,"[оставляет, место, разгадывания, загадок, хотя..."
19996,"[сериал, тех, поиске, квики, секса, обязательс..."
19997,"[вообщето, детективы, боевики, люблю, мелодрам..."
19998,"[изменить, жизнь, помощью, джакузи, плохая, ид..."


In [45]:
sentences = [row for row in df_clean['text']]
word_freq = defaultdict(int)
for sent in sentences:
    if sent != None:
        for i in sent:
            word_freq[i] += 1
        
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['это', 'очень', 'фильм', 'книга', '»', '«', 'просто', '—', 'книги', 'читать']

In [47]:
import sys
w2v_model = Word2Vec(min_count=200,
                     window=5,
                     size=100,
                     workers=5)
                     
w2v_model.build_vocab(sentences)
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=w2v_model.iter)
w2v_model.init_sims(replace=True)

/home/sultan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


In [48]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [52]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='russian'):
        for word in nltk.word_tokenize(sent, language='russian'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens
    
train, test = train_test_split(df, test_size=0.3, random_state = 42)

test_tokenized = test.apply(lambda r: w2v_tokenize_text(r['text']), axis=1).values
train_tokenized = train.apply(lambda r: w2v_tokenize_text(r['text']), axis=1).values

X_train_word_average = word_averaging_list(w2v_model.wv,train_tokenized)
X_test_word_average = word_averaging_list(w2v_model.wv,test_tokenized)

/home/sultan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.vectors_norm instead).
  


In [55]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_word_average, train['score'])
y_pred = logreg.predict(X_test_word_average)
print('accuracy %s' % accuracy_score(y_pred, test.score))
print(classification_report(test.score, y_pred,target_names=my_tags))

/home/sultan/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/sultan/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


accuracy 0.2795
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       106
           2       0.00      0.00      0.00       106
           3       0.20      0.01      0.02       168
           4       0.00      0.00      0.00       265
           5       0.13      0.02      0.04       389
           6       0.25      0.00      0.01       305
           7       0.14      0.02      0.04       576
           8       0.20      0.15      0.17      1056
           9       0.23      0.27      0.25      1427
          10       0.33      0.69      0.44      1602

    accuracy                           0.28      6000
   macro avg       0.15      0.12      0.10      6000
weighted avg       0.22      0.28      0.22      6000



/home/sultan/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
